# Step #01 Importing Packages

In [1]:
from __future__ import division
from datetime import datetime, timedelta,date
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

#import Keras
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras.layers import LSTM
from sklearn.model_selection import KFold, cross_val_score, train_test_split

Using TensorFlow backend.


# Step #02 Fetching Data

In [32]:
data = pd.read_csv('train.csv')
data.head(50)

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
5,2013-01-06,1,1,12
6,2013-01-07,1,1,10
7,2013-01-08,1,1,9
8,2013-01-09,1,1,12
9,2013-01-10,1,1,9


In [33]:
data = data[(data.store==1) & (data.item==1)]

# Step #03 Data Exploration

In [34]:
data.shape

(1826, 4)

In [35]:
data.columns

Index(['date', 'store', 'item', 'sales'], dtype='object')

In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1826 entries, 0 to 1825
Data columns (total 4 columns):
date     1826 non-null object
store    1826 non-null int64
item     1826 non-null int64
sales    1826 non-null int64
dtypes: int64(3), object(1)
memory usage: 71.3+ KB


In [37]:
data.describe()

,store,item,sales
count,1826.0,1826.0,1826.000000
mean,1.0,1.0,19.971522
std,0.0,0.0,6.741022
min,1.0,1.0,4.000000
25%,1.0,1.0,15.000000
50%,1.0,1.0,19.000000
75%,1.0,1.0,24.000000
max,1.0,1.0,50.000000


In [38]:
data.date.min()

'2013-01-01'

In [39]:
data.date.max()

'2017-12-31'

# Step #04 Data Transformation

## Transforming sales data first

* __Convert date field from object to datetime__

In [40]:
#convert date field from string to datetime
data['date'] = pd.to_datetime(data['date'])

#show first 10 rows
data.head(10)

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
5,2013-01-06,1,1,12
6,2013-01-07,1,1,10
7,2013-01-08,1,1,9
8,2013-01-09,1,1,12
9,2013-01-10,1,1,9


* __sorting the whole dataframe on the basis of column "date"__

In [41]:
# sorting the whole dataframe on the basis of column "date"
data = data.sort_values(by=['date'], ascending=True)
# resetting index of the dataframe
data.reset_index(inplace=True)
# dropping unwanted column created while resetting index of the dataframe
data.drop(columns=['index'], inplace=True)
data.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [42]:
# Verfying Changes
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1826 entries, 0 to 1825
Data columns (total 4 columns):
date     1826 non-null datetime64[ns]
store    1826 non-null int64
item     1826 non-null int64
sales    1826 non-null int64
dtypes: datetime64[ns](1), int64(3)
memory usage: 57.1 KB


In [43]:
# Cloning dataframae
df_sales = data.copy(deep=True)

#represent month in date field as its first day
df_sales['date'] = df_sales['date'].dt.year.astype('str') + '-' + df_sales['date'].dt.month.astype('str') + '-01'
# df_sales['date'] = df_sales['date'].dt.year.astype('str') + '-' + df_sales['date'].dt.month.astype('str') + '-' + df_sales['date'].dt.dayofweek.astype('str')

# Converting  column to datatime from string
df_sales['date'] = pd.to_datetime(df_sales['date'])
df_sales

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-01,1,1,11
2,2013-01-01,1,1,14
3,2013-01-01,1,1,13
4,2013-01-01,1,1,10
5,2013-01-01,1,1,12
6,2013-01-01,1,1,10
7,2013-01-01,1,1,9
8,2013-01-01,1,1,12
9,2013-01-01,1,1,9


In [44]:
df_sales.shape

(1826, 4)

In [45]:
#groupby date and sum the sales
df_sales = df_sales.groupby('date').sales.sum().reset_index()

In [46]:
df_sales.shape

(60, 2)

In [47]:
df_sales.columns

Index(['date', 'sales'], dtype='object')

In [48]:
df_sales.date.min()

Timestamp('2013-01-01 00:00:00')

In [49]:
df_sales.date.max()

Timestamp('2017-12-01 00:00:00')

In [50]:
df_sales = df_sales.iloc[:56]
df_sales

,date,sales
0,2013-01-01,328
1,2013-02-01,322
2,2013-03-01,477
3,2013-04-01,522
4,2013-05-01,531
5,2013-06-01,627
6,2013-07-01,661
7,2013-08-01,594
8,2013-09-01,519
9,2013-10-01,484


## Now transforming production data

In [51]:
# Production data
prod_dt = pd.read_csv('production.csv')
prod_dt['observation_date'] = pd.to_datetime(prod_dt['observation_date'])
# sorting the whole dataframe on the basis of column "date"
prod_dt = prod_dt.sort_values(by=['observation_date'], ascending=True)
prod_dt.head()

,observation_date,IPG3113N
0,1972-01-01,85.6945
1,1972-02-01,71.8200
2,1972-03-01,66.0229
3,1972-04-01,64.5645
4,1972-05-01,65.0100


In [52]:
prod_dt.observation_date.min()

Timestamp('1972-01-01 00:00:00')

In [53]:
prod_dt.observation_date.max()

Timestamp('2017-08-01 00:00:00')

In [54]:
prod_dt = prod_dt.iloc[492:]
prod_dt.reset_index(inplace=True)
prod_dt.drop('index', axis=1, inplace=True)
prod_dt

,observation_date,IPG3113N
0,2013-01-01,107.0733
1,2013-02-01,102.0263
2,2013-03-01,102.6319
3,2013-04-01,95.3206
4,2013-05-01,91.7584
5,2013-06-01,91.8125
6,2013-07-01,92.4299
7,2013-08-01,100.3593
8,2013-09-01,105.5167
9,2013-10-01,117.3458


## Merging data

In [55]:
df_sales['production'] = prod_dt['IPG3113N']
# verfying
df_sales

,date,sales,production
0,2013-01-01,328,107.0733
1,2013-02-01,322,102.0263
2,2013-03-01,477,102.6319
3,2013-04-01,522,95.3206
4,2013-05-01,531,91.7584
5,2013-06-01,627,91.8125
6,2013-07-01,661,92.4299
7,2013-08-01,594,100.3593
8,2013-09-01,519,105.5167
9,2013-10-01,484,117.3458


In [56]:
df_sales.production = df_sales.production.round()
df_sales

,date,sales,production
0,2013-01-01,328,107.0
1,2013-02-01,322,102.0
2,2013-03-01,477,103.0
3,2013-04-01,522,95.0
4,2013-05-01,531,92.0
5,2013-06-01,627,92.0
6,2013-07-01,661,92.0
7,2013-08-01,594,100.0
8,2013-09-01,519,106.0
9,2013-10-01,484,117.0


# Step #05: Checking for staionarity in data and moving for transformation if required

In [60]:
#create a new dataframe to model the difference
df_diff = df_sales.copy()
#add previous production to the next row
df_diff['prev_prod'] = df_diff['production'].shift(1)
#drop the null values and calculate the difference
df_diff = df_diff.dropna()
df_diff['diff'] = (df_diff['production'] - df_diff['prev_prod'])
df_diff.head(10)

,date,sales,production,prev_prod,diff
1,2013-02-01,322,102.0,107.0,-5.0
2,2013-03-01,477,103.0,102.0,1.0
3,2013-04-01,522,95.0,103.0,-8.0
4,2013-05-01,531,92.0,95.0,-3.0
5,2013-06-01,627,92.0,92.0,0.0
6,2013-07-01,661,92.0,92.0,0.0
7,2013-08-01,594,100.0,92.0,8.0
8,2013-09-01,519,106.0,100.0,6.0
9,2013-10-01,484,117.0,106.0,11.0
10,2013-11-01,572,122.0,117.0,5.0


In [61]:
#plot production diff
plot_data = [
    go.Scatter(
        x=df_diff['date'],
        y=df_diff['diff'],
    )
]
plot_layout = go.Layout(
        title='Montly production Diff'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
py.iplot(fig)

In [62]:
#create dataframe for transformation from time series to supervised
df_supervised = df_diff.drop(['prev_prod'],axis=1)
#adding lags
for inc in range(1,13):
    field_name = 'lag_' + str(inc)
    df_supervised[field_name] = df_supervised['diff'].shift(inc)
#drop null values
df_supervised = df_supervised.dropna().reset_index(drop=True)

In [63]:
df_supervised

,date,sales,production,diff,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12
0,2014-02-01,405,104.0,-1.0,-18.0,1.0,5.0,11.0,6.0,8.0,0.0,0.0,-3.0,-8.0,1.0,-5.0
1,2014-03-01,549,101.0,-3.0,-1.0,-18.0,1.0,5.0,11.0,6.0,8.0,0.0,0.0,-3.0,-8.0,1.0
2,2014-04-01,573,93.0,-8.0,-3.0,-1.0,-18.0,1.0,5.0,11.0,6.0,8.0,0.0,0.0,-3.0,-8.0
3,2014-05-01,651,88.0,-5.0,-8.0,-3.0,-1.0,-18.0,1.0,5.0,11.0,6.0,8.0,0.0,0.0,-3.0
4,2014-06-01,710,89.0,1.0,-5.0,-8.0,-3.0,-1.0,-18.0,1.0,5.0,11.0,6.0,8.0,0.0,0.0
5,2014-07-01,746,88.0,-1.0,1.0,-5.0,-8.0,-3.0,-1.0,-18.0,1.0,5.0,11.0,6.0,8.0,0.0
6,2014-08-01,671,98.0,10.0,-1.0,1.0,-5.0,-8.0,-3.0,-1.0,-18.0,1.0,5.0,11.0,6.0,8.0
7,2014-09-01,566,107.0,9.0,10.0,-1.0,1.0,-5.0,-8.0,-3.0,-1.0,-18.0,1.0,5.0,11.0,6.0
8,2014-10-01,555,120.0,13.0,9.0,10.0,-1.0,1.0,-5.0,-8.0,-3.0,-1.0,-18.0,1.0,5.0,11.0
9,2014-11-01,619,129.0,9.0,13.0,9.0,10.0,-1.0,1.0,-5.0,-8.0,-3.0,-1.0,-18.0,1.0,5.0


# Step #06 Applying Model LSTM

In [64]:
# Import statsmodels.formula.api
import statsmodels.formula.api as smf
# Define the regression formula
model = smf.ols(formula='diff ~ lag_1', data=df_supervised)
# Fit the regression
model_fit = model.fit()
# Extract the adjusted r-squared
regression_adj_rsq = model_fit.rsquared_adj
print(regression_adj_rsq)

0.034594611501579386


In [65]:
df_supervised.columns

Index(['date', 'sales', 'production', 'diff', 'lag_1', 'lag_2', 'lag_3',
       'lag_4', 'lag_5', 'lag_6', 'lag_7', 'lag_8', 'lag_9', 'lag_10',
       'lag_11', 'lag_12'],
      dtype='object')

In [66]:
#import MinMaxScaler and create a new dataframe for LSTM model
from sklearn.preprocessing import MinMaxScaler
df_model = df_supervised.drop(['sales','date', 'production'],axis=1)
#split train and test set
train_set, test_set = df_model[0:-6].values, df_model[-6:].values

In [67]:
#apply Min Max Scaler
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(train_set)
# reshape training set
train_set = train_set.reshape(train_set.shape[0], train_set.shape[1])
train_set_scaled = scaler.transform(train_set)
# reshape test set
test_set = test_set.reshape(test_set.shape[0], test_set.shape[1])
test_set_scaled = scaler.transform(test_set)

In [68]:
X_train, y_train = train_set_scaled[:, 1:], train_set_scaled[:, 0:1]
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test, y_test = test_set_scaled[:, 1:], test_set_scaled[:, 0:1]
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [69]:
model = Sequential()
model.add(LSTM(4, batch_input_shape=(1, X_train.shape[1], X_train.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, nb_epoch=100, batch_size=1, verbose=1, shuffle=False)

Epoch 1/100
37/37 [==============================] - 2s 44ms/step - loss: 0.1361
Epoch 2/100
37/37 [==============================] - 0s 2ms/step - loss: 0.1246
Epoch 3/100
37/37 [==============================] - 0s 2ms/step - loss: 0.1163
Epoch 4/100
37/37 [==============================] - 0s 1ms/step - loss: 0.1093
Epoch 5/100
37/37 [==============================] - 0s 2ms/step - loss: 0.1030
Epoch 6/100
37/37 [==============================] - 0s 2ms/step - loss: 0.0976
Epoch 7/100
37/37 [==============================] - 0s 2ms/step - loss: 0.0928
Epoch 8/100
37/37 [==============================] - 0s 1ms/step - loss: 0.0886
Epoch 9/100
37/37 [==============================] - 0s 2ms/step - loss: 0.0850
Epoch 10/100
37/37 [==============================] - 0s 1ms/step - loss: 0.0819
Epoch 11/100
37/37 [==============================] - 0s 1ms/step - loss: 0.0792
Epoch 12/100
37/37 [==============================] - 0s 2ms/step - loss: 0.0768
Epoch 13/100
37/37 [================

In [70]:
y_pred = model.predict(X_test,batch_size=1)
#for multistep prediction, you need to replace X_test values with the predictions coming from t-1

In [71]:
#reshape y_pred
y_pred = y_pred.reshape(y_pred.shape[0], 1, y_pred.shape[1])
#rebuild test set for inverse transform
pred_test_set = []
for index in range(0,len(y_pred)):
    print(np.concatenate([y_pred[index],X_test[index]],axis=1))
    pred_test_set.append(np.concatenate([y_pred[index],X_test[index]],axis=1))
#reshape pred_test_set
pred_test_set = np.array(pred_test_set)
pred_test_set = pred_test_set.reshape(pred_test_set.shape[0], pred_test_set.shape[2])
#inverse transform
pred_test_set_inverted = scaler.inverse_transform(pred_test_set)

[[-0.20281619  0.4375     -0.25        0.125       0.0625      0.8125
   0.4375      0.3125      0.25        0.1875      0.0625     -0.0625
   0.1875    ]]
[[ 0.15323068 -0.3125      0.4375     -0.25        0.125       0.0625
   0.8125      0.4375      0.3125      0.25        0.1875      0.0625
  -0.0625    ]]
[[ 0.02506004  0.3125     -0.3125      0.4375     -0.25        0.125
   0.0625      0.8125      0.4375      0.3125      0.25        0.1875
   0.0625    ]]
[[ 0.30067578 -0.125       0.3125     -0.3125      0.4375     -0.25
   0.125       0.0625      0.8125      0.4375      0.3125      0.25
   0.1875    ]]
[[ 0.00729957  0.3125     -0.125       0.3125     -0.3125      0.4375
  -0.25        0.125       0.0625      0.8125      0.4375      0.3125
   0.25      ]]
[[ 0.48386547  0.125       0.3125     -0.125       0.3125     -0.3125
   0.4375     -0.25        0.125       0.0625      0.8125      0.4375
   0.3125    ]]


In [72]:
#create dataframe that shows the predicted sales
result_list = []
sales_dates = list(df_sales[-7:].date)
act_sales = list(df_sales[-7:].sales)
for index in range(0,len(pred_test_set_inverted)):
    result_dict = {}
    result_dict['pred_value'] = int(pred_test_set_inverted[index][0] + act_sales[index])
    result_dict['date'] = sales_dates[index+1]
    result_list.append(result_dict)
df_result = pd.DataFrame(result_list)
#for multistep prediction, replace act_sales with the predicted sales

In [73]:
df_result

,date,pred_value
0,2017-03-01,480
1,2017-04-01,560
2,2017-05-01,674
3,2017-06-01,774
4,2017-07-01,847
5,2017-08-01,877


In [74]:
#merge with actual sales dataframe
df_sales_pred = pd.merge(df_sales,df_result,on='date',how='left')
#plot actual and predicted
plot_data = [
    go.Scatter(
        x=df_sales_pred['date'],
        y=df_sales_pred['sales'],
        name='actual'
    ),
        go.Scatter(
        x=df_sales_pred['date'],
        y=df_sales_pred['pred_value'],
        name='predicted'
    )
    
]
plot_layout = go.Layout(
        title='Sales Prediction'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
py.iplot(fig)